## Preparation of r/news data by Nicholas Osaka

In [1]:
import numpy as np
import pandas as pd
import nltk
import sklearn as sk

In [2]:
df = pd.read_csv('../datasets/news-20k-comments.csv')
df.head()

,created_utc,ups,subreddit_id,link_id,name,score_hidden,author_flair_css_class,author_flair_text,subreddit,id,...,downs,archived,author,score,retrieved_on,body,distinguished,edited,controversiality,parent_id
0,1430438402,-11.0,t5_2qh3l,t3_34f1lq,t1_cqug92b,0.0,NaN,NaN,news,cqug92b,...,0.0,0.0,hogsucker,-11.0,1.432703e+09,1-She got to be a bigwig at Google by sleeping...,NaN,0.0,0.0,t1_cqu4t11
1,1430438407,1.0,t5_2qh3l,t3_34exjb,t1_cqug96h,0.0,NaN,NaN,news,cqug96h,...,0.0,0.0,flal4,1.0,1.432703e+09,For those about to lynch this guy [here](http:...,NaN,0.0,1.0,t1_cqudz0p
2,1430438439,4.0,t5_2qh3l,t3_34f10p,t1_cqug9tk,0.0,NaN,NaN,news,cqug9tk,...,0.0,0.0,HitachinoBia,4.0,1.432703e+09,It feels like black people are the most racist...,NaN,0.0,1.0,t1_cqufsip
3,1430438448,0.0,t5_2qh3l,t3_34cvvg,t1_cquga1l,0.0,NaN,NaN,news,cquga1l,...,0.0,0.0,[deleted],0.0,1.432703e+09,[deleted],NaN,0.0,0.0,t3_34cvvg
4,1430438449,-10.0,t5_2qh3l,t3_34e7eo,t1_cquga1v,0.0,NaN,NaN,news,cquga1v,...,0.0,0.0,Cultiststeve,-10.0,1.432703e+09,Its because otherwise thats all that would app...,NaN,0.0,0.0,t1_cqudxkr


In [3]:
all_comments = df['body'] # put all the comment bodies into a pandas DataFrame
all_comments.describe()

count         20000
unique        17491
top       [deleted]
freq           2468
Name: body, dtype: object

We can see that the most frequent comment is "[deleted]" which we need to remove.

In [4]:
# deleted comments Series
deleted_comments = all_comments[all_comments == "[deleted]"]
deleted_comments.describe()

count          2468
unique            1
top       [deleted]
freq           2468
Name: body, dtype: object

In [5]:
# valid comments!
comments = all_comments.drop(deleted_comments.index)
comments = comments.to_frame().reset_index(drop=True)
comments.head()

,body
0,1-She got to be a bigwig at Google by sleeping...
1,For those about to lynch this guy [here](http:...
2,It feels like black people are the most racist...
3,Its because otherwise thats all that would app...
4,Please go to Facebook and comment and post on ...


In [12]:
# split into tokens

'Please go to Facebook and comment and post on the "Fire Wyckoff Police Officer Kyle Ferreira" Facebook page!! Not many have commented yet. What else can we d\no to help?! '